In [1]:
import os
import numpy as np
import scipy.stats as sps
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import prune
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import pickle
from utils import get_network_tinyimagenet
from conf import settings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import quant_utils
import copy
import random
seed = 0

gpu = 'cuda:0'
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic=True

In [2]:
subset1_mean, subset1_std = settings.TINYIMAGENET_SUBTRAIN_MEAN[0], settings.TINYIMAGENET_SUBTRAIN_STD[0]

def load_model(path, norm=False, dev='cpu'):
    vic = get_network_tinyimagenet('mobilenet_v2', False).to(dev)
    vic.load_state_dict(torch.load(path, map_location=dev))
    vic.eval()
    if norm:
        return nn.Sequential(transforms.Normalize(subset1_mean, subset1_std), vic)
    return vic

In [3]:
def global_prune(net, p=0.2):
    copy_net = copy.deepcopy(net)
    parameters_to_prune = []
    for name, module in copy_net.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            parameters_to_prune.append((module, 'weight'))
        if isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))
    prune.global_unstructured(
        tuple(parameters_to_prune),
        pruning_method=prune.L1Unstructured,
        amount=p,
    )
    return copy_net

# Paths

In [4]:
root_path = '/data1/checkpoint/'
related_folder_path = os.path.join(root_path, 'hash/tinyimagenet/')
unrelated_folder_path = os.path.join(root_path, 'hash/tinyimagenet/independent')

original_path = os.path.join(related_folder_path, 'mobilenet_v2_0.pth')
quant_path = os.path.join(related_folder_path, 'mobilenet_v2_0_quant.pth')

finetune_path_dict = {}dt19
for fid in range(5):
    folder_name = 'finetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 11)]
    folder_name = 'advfinetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 11)]

for fid in range(5, 10):
    folder_name = 'finetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 21)]
    folder_name = 'advfinetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 21)]

unrelated_path_list = [os.path.join(unrelated_folder_path, 'model_{}.pth'.format(i)) for i in range(200)]

In [5]:
norm = True
n = 1000
input_shape = (n, ) + (3, 64, 64)
# checkpoint_out = {}
output_dict = {}
randf = torch.rand

In [6]:
with torch.no_grad():
    # original net
    original_net = load_model(original_path, norm=norm, dev=gpu)
    output_dict['train'] = original_net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu')

    # quant net
    net = quant_utils.load_torchscript_model(quant_path, 'cpu')
    if norm:
        net = nn.Sequential(transforms.Normalize(subset1_mean, subset1_std), net)
    output_dict['quant'] = net(randf(input_shape)).softmax(dim=1)
    
    # pruning
    output_dict['prune'] = {}
    for prune_p in [0.8, 0.6, 0.4, 0.2]:
        net = global_prune(original_net, prune_p)
        output_dict['prune'][prune_p] = net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu')
    
    # finetuned model
    output_dict['finetune'] = {}
    for name, pathlist in tqdm(finetune_path_dict.items()):
        output_dict['finetune'][name] = []
        for path in pathlist:
            net = load_model(path, norm=norm, dev=gpu)
            output_dict['finetune'][name].append(net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu'))

    # finetuned model
    output_dict['indep'] = []
    for i, path in tqdm(enumerate(unrelated_path_list)):
        net = load_model(path, norm=norm, dev=gpu)
        output_dict['indep'].append(net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu'))

100%|██████████| 20/20 [01:37<00:00,  4.88s/it]
200it [01:21,  2.46it/s]


# Train autoencoder

In [7]:

autoencoder = nn.Sequential(
                nn.Linear(200, 128),
                nn.ReLU(),
                nn.Linear(128, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 128),
                nn.ReLU(),
                nn.Linear(128, 128),
                nn.ReLU(),
                nn.Linear(128, 200),
            ).to(gpu)
ae_dataset = torch.utils.data.TensorDataset(output_dict['train'], torch.zeros(len(output_dict['train'])).long()) 
ae_train_dataloader = torch.utils.data.DataLoader(ae_dataset, 
                                                  batch_size=8, 
                                                  shuffle=True, num_workers=8)
n_epoch = 50
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
criterion = nn.MSELoss()

with tqdm(total=n_epoch, desc="train") as pbar:
    for epoch in range(n_epoch):
        epoch_loss = 0
        for x, y in ae_train_dataloader:
            x = x.to(gpu)
            outx = autoencoder(x).softmax(dim=1)
            loss = criterion(x, outx)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * len(y)
#         print(epoch, epoch_loss / len(ae_dataset))
        pbar.set_postfix({'loss' : '{0:1.10e}'.format(epoch_loss / len(ae_dataset))}) 
        pbar.update(1)
autoencoder.to('cpu')

train: 100%|██████████| 50/50 [00:28<00:00,  1.75it/s, loss=2.2941999441e-06]


Sequential(
  (0): Linear(in_features=200, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=128, bias=True)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=200, bias=True)
)

In [8]:
vic_errors = []
with torch.no_grad():
    for i in range(1):
        randout = original_net(randf(input_shape, device=gpu)).softmax(dim=1).cpu()
        out = autoencoder(randout).softmax(dim=1)
        vic_errors = torch.sum((randout - out)**2, dim=1).numpy()
        print('{}, {:.2e}, {:.2e}'.format(i, np.mean(vic_errors), np.std(vic_errors)))

0, 4.57e-04, 1.47e-03


In [9]:
with torch.no_grad():
    out = autoencoder(output_dict['train']).softmax(dim=1)
    errors = torch.sum((output_dict['train'] - out)**2, dim=1).numpy()
    print(sps.ks_2samp(errors, vic_errors))
    print("{:.2e}, {:.2e}".format(np.mean(errors), np.std(errors)))

KstestResult(statistic=0.034, pvalue=0.6101664688189142)
4.33e-04, 1.06e-03


In [10]:
with torch.no_grad():
    
    out = autoencoder(output_dict['quant']).softmax(dim=1)
    errors = torch.sum((output_dict['quant'] - out)**2, dim=1).numpy()
    print(sps.ks_2samp(errors, vic_errors))
    print("{:.2e}, {:.2e}".format(np.mean(errors), np.std(errors)))

KstestResult(statistic=0.067, pvalue=0.022438659451142425)
5.51e-04, 1.78e-03


In [11]:
with torch.no_grad():
    for k, v in output_dict['prune'].items():
        print(k)
        out = autoencoder(v).softmax(dim=1)
        errors = torch.sum((v - out)**2, dim=1).numpy()
        stats, pv = sps.ks_2samp(errors, vic_errors)
        print("{:.2e}, {:.2e}, {:.2e}, {:.2e}".format(
            np.mean(errors), np.std(errors), stats, pv))
        

0.8
6.86e-03, 7.31e-03, 9.45e-01, 0.00e+00
0.6
5.14e-04, 1.49e-03, 4.50e-02, 2.63e-01
0.4
5.18e-04, 1.35e-03, 7.70e-02, 5.31e-03
0.2
4.17e-04, 7.68e-04, 5.00e-02, 1.64e-01


In [12]:
finetune_errors = {}
indep_errors = []

In [13]:
with torch.no_grad():
    for k, vlist in output_dict['finetune'].items():
        print(k)
        finetune_errors[k] = []
        for i, v in enumerate(vlist):
            out = autoencoder(v).softmax(dim=1)
            errors = torch.sum((v - out)**2, dim=1).numpy()
            stats, pv = sps.ks_2samp(errors, vic_errors)
            finetune_errors[k].append(errors)
            print(i, "{:.2e}, {:.2e}, {:.2e}, {:.2e}".format(
            np.mean(errors), np.std(errors), stats, pv))

finetune_0
0 8.30e-04, 6.05e-03, 5.60e-02, 8.69e-02
1 5.27e-04, 1.70e-03, 5.10e-02, 1.48e-01
2 5.83e-04, 1.92e-03, 3.40e-02, 6.10e-01
3 6.36e-04, 1.82e-03, 6.70e-02, 2.24e-02
4 7.42e-04, 3.70e-03, 9.30e-02, 3.48e-04
5 9.95e-04, 3.31e-03, 1.68e-01, 9.80e-13
6 8.43e-04, 3.17e-03, 1.55e-01, 6.76e-11
7 8.11e-04, 3.24e-03, 1.40e-01, 5.82e-09
8 1.41e-03, 5.46e-03, 2.65e-01, 2.83e-31
9 1.48e-03, 6.47e-03, 2.15e-01, 1.20e-20
advfinetune_0
0 1.03e-03, 4.39e-03, 1.87e-01, 1.08e-15
1 8.59e-04, 2.95e-03, 1.46e-01, 1.04e-09
2 8.12e-04, 2.88e-03, 1.47e-01, 7.71e-10
3 5.75e-04, 2.30e-03, 3.80e-02, 4.66e-01
4 5.93e-04, 1.71e-03, 8.90e-02, 7.21e-04
5 1.08e-03, 4.22e-03, 2.02e-01, 2.93e-18
6 7.49e-04, 2.41e-03, 1.37e-01, 1.34e-08
7 8.08e-04, 3.15e-03, 1.39e-01, 7.71e-09
8 2.45e-03, 1.35e-02, 3.11e-01, 4.10e-43
9 1.32e-03, 4.51e-03, 2.26e-01, 8.66e-23
finetune_1
0 8.57e-04, 3.69e-03, 9.90e-02, 1.10e-04
1 6.33e-04, 2.12e-03, 5.80e-02, 6.92e-02
2 6.25e-04, 2.46e-03, 8.20e-02, 2.39e-03
3 4.76e-04, 1.09e-03,

In [14]:
with torch.no_grad():
    for i, v in enumerate(output_dict['indep']):
        out = autoencoder(v).softmax(dim=1)
        errors = torch.sum((v - out)**2, dim=1).numpy()
        indep_errors.append(errors)
        stats, pv = sps.ks_2samp(errors, vic_errors)
        print(i, "{:.2e}, {:.2e}, {:.2e}, {:.2e}".format(
            np.mean(errors), np.std(errors), stats, pv))

0 6.98e-03, 7.81e-03, 8.55e-01, 0.00e+00
1 9.04e-03, 9.06e-03, 9.04e-01, 0.00e+00
2 7.54e-03, 8.00e-03, 9.03e-01, 0.00e+00
3 1.96e-02, 1.70e-02, 9.32e-01, 0.00e+00
4 2.93e-02, 1.83e-02, 9.83e-01, 0.00e+00
5 2.61e-02, 2.16e-02, 9.62e-01, 0.00e+00
6 9.55e-02, 7.30e-02, 9.81e-01, 0.00e+00
7 8.10e-02, 7.64e-02, 9.77e-01, 0.00e+00
8 2.01e-02, 1.87e-02, 9.26e-01, 0.00e+00
9 4.82e-03, 8.89e-03, 7.58e-01, 7.24e-282
10 1.18e-01, 4.63e-02, 9.97e-01, 0.00e+00
11 1.42e-02, 1.77e-02, 9.37e-01, 0.00e+00
12 4.35e-02, 4.12e-02, 9.72e-01, 0.00e+00
13 6.02e-01, 1.72e-01, 1.00e+00, 0.00e+00
14 2.03e-01, 1.17e-01, 9.94e-01, 0.00e+00
15 4.80e-02, 2.00e-02, 9.96e-01, 0.00e+00
16 1.56e-01, 5.67e-02, 1.00e+00, 0.00e+00
17 9.74e-02, 6.73e-02, 9.95e-01, 0.00e+00
18 1.45e-01, 1.06e-01, 9.92e-01, 0.00e+00
19 1.59e-01, 8.65e-02, 9.98e-01, 0.00e+00
20 8.61e-02, 8.20e-02, 9.85e-01, 0.00e+00
21 1.15e-01, 7.52e-02, 9.98e-01, 0.00e+00
22 1.04e-01, 5.84e-02, 9.92e-01, 0.00e+00
23 1.66e-01, 7.73e-02, 9.99e-01, 0.00e+00
2

In [15]:
torch.save(autoencoder.state_dict(), '../results/hash/tinyimagenet/autoencoder.pt')

pickle.dump((output_dict, finetune_errors, indep_errors), open("../results/hash/tinyimagenet/output_dict.pkl", "wb"))

In [16]:
finetune_mean_errors = []
for k, vlist in finetune_errors.items():
    if int(k.split('_')[-1])<5:
        for v in vlist:
            finetune_mean_errors.append(np.mean(v))
print(np.max(finetune_mean_errors))

0.00245189


In [17]:
test_finetune_mean_errors = []
for k, vlist in finetune_errors.items():
    if int(k.split('_')[-1])>4:
        for v in vlist[:20]:
            
            test_finetune_mean_errors.append(np.mean(v))

In [18]:
test_finetune_mean_errors = np.array(test_finetune_mean_errors)
print(len(test_finetune_mean_errors[test_finetune_mean_errors>np.max(finetune_mean_errors)]))

1


In [19]:
test_indep_mean_errors = []
for v in indep_errors[:200]:
    test_indep_mean_errors.append(np.mean(v))
test_indep_mean_errors = np.array(test_indep_mean_errors)
print(len(test_indep_mean_errors[test_indep_mean_errors<np.max(finetune_mean_errors)]))

0


In [20]:
# from utils import compute_accuracy, get_test_dataloader_tinyimagenet
# test_loader = get_test_dataloader_tinyimagenet((0, 0, 0), (1, 1, 1))
# acc, _, _ = compute_accuracy(original_net, test_loader, 'cpu')
# accquant, _, _ = compute_accuracy(quant_net, test_loader, 'cpu')
# acc20, _, _ = compute_accuracy(pruned_model_2, test_loader, 'cpu')
# acc40, _, _ = compute_accuracy(pruned_model_4, test_loader, 'cpu')
# acc60, _, _ = compute_accuracy(pruned_model_6, test_loader, 'cpu')
# acc80, _, _ = compute_accuracy(pruned_model_8, test_loader, 'cpu')
# print(acc, accquant, acc20, acc40, acc60, acc80)